<a href="https://colab.research.google.com/github/anindita/dances/blob/main/detect/matlab_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp --verbose -R /content/drive/MyDrive/mpii/annot /content/

import json
import random

valid = []
test = []
train = []

with open('annot/ExPose3000Diving.json') as json_data:
  data = json.load(json_data)
  for img in data:
    rand = random.randint(1, 20)
    if rand == 1 or rand == 2 or rand == 3:
      valid.append(img)
    elif rand == 4 or rand == 5 or rand == 6:
      test.append(img)
    else:
      train.append(img)

with open('valid.json', 'w') as f:
    json.dump(valid, f, ensure_ascii=False, indent=4)

with open('test.json', 'w') as f:
    json.dump(test, f, ensure_ascii=False, indent=4)

with open('train.json', 'w') as f:
    json.dump(train, f, ensure_ascii=False, indent=4)

In [ ]:
from scipy.io import loadmat, savemat

gt_dict = loadmat('/content/gt_valid.mat')

dataset_joints = gt_dict['dataset_joints']
jnt_missing = gt_dict['jnt_missing']
pos_gt_src = gt_dict['pos_gt_src']
headboxes_src = gt_dict['headboxes_src']

In [ ]:
choice = headboxes_src

print(choice)
print(len(choice))

In [ ]:
## Missing joint data

with open('/content/test.json') as json_data:
  data = json.load(json_data)

  joint_missing = [[] for joint in data[0]['joints_vis']]

  for i,img in enumerate(data):
    for j,jnt in enumerate(img['joints_vis']):
      if jnt == 0:
        joint_missing[j].append(1)
      else:
        joint_missing[j].append(0)

print(joint_missing)


In [ ]:
## Positive ground truth sources

with open('/content/test.json') as json_data:
  data = json.load(json_data)

  ground_truth = [[[] for j in jnt] for jnt in data[0]['joints']]

  for i,img in enumerate(data):
    for j,jnt in enumerate(img['joints']): # 16 joints
      ground_truth[j][0].append(jnt[0])
      ground_truth[j][1].append(jnt[1])

print(ground_truth)

In [ ]:
## Headboxes

import math

with open('/content/test.json') as json_data:
  data = json.load(json_data)

  heads = [[[] for coord in range(2)] for jnt in range(2)]
  # bttm l x, bttm l y, top r x, top r y

  for i,img in enumerate(data):
    neck_x = img['joints'][8][0]
    neck_y = img['joints'][8][1]
    head_x = img['joints'][9][0]
    head_y = img['joints'][9][1]

    half = math.sqrt((neck_x - head_x) ** 2 + (neck_y - head_y) ** 2) / 2

    mid_x = (neck_x + head_x) / 2
    mid_y = (neck_y + head_y) / 2

    bml_x = int(min(mid_x - half, neck_x, head_x))
    bml_y = int(min(mid_y - half, neck_y, head_y))
    tpr_x = int(max(mid_x + half, neck_x, head_x))
    tpr_y = int(max(mid_y + half, neck_y, head_y))

    heads[0][0].append(bml_x)
    heads[0][1].append(bml_y)
    heads[1][0].append(tpr_x)
    heads[1][1].append(tpr_y)

print(heads)

In [ ]:
savemat("gt_valid.mat", mdict={'dataset_joints': dataset_joints, 'jnt_missing': joint_missing, 'pos_gt_src': ground_truth, 'headboxes_src': heads})